# 텍스트 분류 2차 시도 (데이터 10,629행)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [60]:
df2 = pd.read_csv('final_text_data.csv', sep='\t', encoding='UTF-8')

In [61]:
df2.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN  
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN  
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN  
3              [C] 현상및조치   차량출고시 웨즈스트립고무 주름발생 들뜸교환조치     X  NaN  NaN  NaN  
4                     [C] 몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고     X  NaN  NaN  NaN

In [62]:
df2.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인'],
      dtype='object')

In [63]:
df2.shape

(10629, 16)

In [64]:
category_count = df2 ['대상 선정'].value_counts()
category_count

X    5519
O    3013
Name: 대상 선정, dtype: int64

In [65]:
category_count = df2 ['책임구'].value_counts()
category_count

구매      2273
생산공장     725
연구소       14
Name: 책임구, dtype: int64

In [66]:
category_count = df2 ['원인'].value_counts()
category_count

부품       2273
조립불량      230
결선불량      225
체결불량      103
기타        101
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# ◈ Null 제거

In [67]:
df_bak = df2.copy()

In [68]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df2 = df2.fillna('N')

In [69]:
df2.shape

(10629, 16)

In [70]:
category_count = df2 ['대상 선정'].value_counts()
category_count

X    5519
O    3013
N    2097
Name: 대상 선정, dtype: int64

In [71]:
category_count = df2 ['책임구'].value_counts()
category_count

N       7617
구매      2273
생산공장     725
연구소       14
Name: 책임구, dtype: int64

In [72]:
category_count = df2 ['원인'].value_counts()
category_count

N        7617
부품       2273
조립불량      230
결선불량      225
체결불량      103
기타        101
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

In [73]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df2 ['target_predict'] = df2 ['대상 선정'].astype(str).copy()
# df2 ['train_charge_predict'] = df2 ['책임구'].astype(str).copy()
# df2 ['train_cause_predict'] = df2 ['원인'].astype(str).copy()

In [74]:
def preprocessing(text):
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)   #-> 특수문자 제거
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)         #-> 한글, 영문, 숫자만 남기고 모두 제거
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)              #-> 한글, 영문만 남기고 모두 제거
#     text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text)              #-> 한글만 남기고 모두 제거
    return text

In [75]:
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [76]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(preprocessing)

Wall time: 159 ms


In [77]:
df2['clm_prtl_sbc'].head()

0     C  현상엔진룸 다량의 연료 누유 발생점검엔진룸 다량의 연료 누유 점검점검내용엔진...
1                             C  현상 리어 도어 앤티 취핑 필림 미장착
2     C  현상Rr RH 커튼 후크  미장착 출고됨조치내용Rr RH 커튼 후크 장착후 출고함
3                 C  현상및조치   차량출고시 웨즈스트립고무 주름발생 들뜸교환조치
4                        C  몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고
Name: clm_prtl_sbc, dtype: object

In [78]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 61 ms


In [79]:
df2['clm_prtl_sbc'].head()

0     C  현상엔진룸 다량의 연료 누유 발생점검엔진룸 다량의 연료 누유 점검점검내용엔진...
1                                C  리어 도어 앤티 취핑 필림 미장착
2     C  현상Rr RH 커튼 후크  미장착 출고됨조치내용Rr RH 커튼 후크 장착후 출고함
3                 C  현상및조치   차량출고시 웨즈스트립고무 주름발생 들뜸교환조치
4                        C  몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고
Name: clm_prtl_sbc, dtype: object

# ◈ Train Data & Test Data 만들기

In [80]:
df2 = df2.reindex()

In [81]:
df2.shape

(10629, 17)

In [82]:
split_count = int(df2.shape[0]*0.9)
split_count

9566

In [83]:
df2_train = df2[:split_count].copy()
df2_train.shape

(9566, 17)

In [84]:
df2_test = df2[split_count:].copy()
df2_test.shape

(1063, 17)

# ◈ 단어 벡터화

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
stops = ['[C]']
vectorizer = CountVectorizer(analyzer = 'word',    # 캐릭터 단위로도 벡터화가능
                             tokenizer = None,     # 토크나이저를 따로 지정해 줄 수도 있음
                             preprocessor = None,  # 전처리 도구
                             stop_words = stops,   # 불용어 nltk등의 도구를 사용할 수도 있음
                             min_df = 1,           # 토큰출현 최소문서개수:오타 및 자주 출현 안하는 특수전문용어 제거에 좋음
                             ngram_range=(1, 3),   # BOW단위를 1~3개로 지정
                             max_features = 20000  # 만들 피처수, 단어수가 됨
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=['[C]'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [86]:
%%time
train_feature_vector = vectorizer.fit_transform(df2_train['clm_prtl_sbc'])
train_feature_vector.shape

Wall time: 1.53 s


In [87]:
%%time
test_feature_vector = vectorizer.fit_transform(df2_test['clm_prtl_sbc'])
test_feature_vector.shape

Wall time: 168 ms


In [88]:
vocab1 = vectorizer.get_feature_names()
print(len(vocab1))
vocab1[:10]

20000


['ab',
 'assy',
 'assy 교환',
 'cm',
 'cs',
 'cs 차량으로',
 'db',
 'db로',
 'db에서',
 'db에서 db로']

In [89]:
dist = np.sum(train_feature_vector, axis=0)
vec_df2 = pd.DataFrame(dist, columns = vocab1)
vec_df2

ab  assy  assy 교환  cm  cs  cs 차량으로  db  db로  db에서  db에서 db로    ...      \
0  46     9        4   5   3        3   3    6     3         5    ...       

   히전정도 돌린상태에서  히전정도 돌린상태에서 출발시  히터  히터 온도  히터 온도 교환시  히터 온도 엑추에이터  히터 작동불량  \
0            3                4   3      5         10            5        5   

   히터 작동불량 발생됨점검리어  히터모드  히터모드 변경불량  
0               31     3          3  

[1 rows x 20000 columns]

In [90]:
vec_df2.columns

Index(['ab', 'assy', 'assy 교환', 'cm', 'cs', 'cs 차량으로', 'db', 'db로', 'db에서',
       'db에서 db로',
       ...
       '히전정도 돌린상태에서', '히전정도 돌린상태에서 출발시', '히터', '히터 온도', '히터 온도 교환시',
       '히터 온도 엑추에이터', '히터 작동불량', '히터 작동불량 발생됨점검리어', '히터모드', '히터모드 변경불량'],
      dtype='object', length=20000)

# ◈ tf-idf 가중치 적용하기

In [91]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer (smooth_idf = False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [92]:
%%time
train_feature_tfidf = transformer.fit_transform (train_feature_vector)
train_feature_tfidf.shape

Wall time: 103 ms


In [93]:
%%time
test_feature_tfidf = transformer.fit_transform (test_feature_vector)
test_feature_tfidf.shape

Wall time: 86 ms


# ◈ Machine Learning

In [94]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier (
    n_estimators = 100, n_jobs = -1, random_state = 2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [95]:
y_label = df2_train['target_predict']
%time forest = forest.fit (train_feature_tfidf, y_label)

Wall time: 4.14 s


# ◈ Model 평가

In [96]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold (n_splits = 5, shuffle = True, random_state = 0)

scoring = 'accuracy'
%time score = cross_val_score (forest, train_feature_vector, y_label, cv = k_fold, n_jobs = -1, scoring = scoring)
score

Wall time: 39.4 s


array([0.60815047, 0.56926294, 0.60951385, 0.6021955 , 0.59749085])

In [97]:
round (np.mean(score)*100,1)

59.7

# ◈ 예측 (Test Data 이용)

In [98]:
y_predict = forest.predict(test_feature_vector)
y_predict[:3]

array(['O', 'X', 'N'], dtype=object)

In [99]:
y_predict.shape

(1063,)

In [100]:
output = pd.DataFrame (data = {'test_category_predict':y_predict})
output.head()

test_category_predict
0                     O
1                     X
2                     N
3                     X
4                     O

In [101]:
df2_test['test_category_predict'] = y_predict

In [102]:
df2_test.head(5)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
9566               W  2017     20171127         ZZ5         Q57   소음/이음_작동시   
9567               W  2015     20150824         ZZ5         Q57   소음/이음_작동시   
9568               W  2015     20150213         ZZ3         Q57   소음/이음_작동시   
9569               W  2016     20161021         ZZ3         Q57   소음/이음_작동시   
9570               W  2016     20161213         ZZ3         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
9566         KV11          CK     J5          TT        82130   
9567         KV11          YP     A9          TT        83940   
9568         KV11          YP     A9          TT        82210   
9569         KV11          YP     A9          TT        83940   
9570         KV11          YP     A9          CC        32800   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  \
9566   C  방지턱 통과시 좌우측 FR도어부근 찌그덕 고무 비비는 소음발생  웨더스트립 ...     N  N   N  N   
9567   C   우측 슬라이드도어 닫힘시 쁘윽 소음 발생점검 및 조치내용  점검결과 리어도...     N  N   N  N   
9568   C    상기차량은 운전석 윈도우 하강 시  끄윽 거리는 이음이 발생하여 입고 됨...     N  N   N  N   
9569   C  JXW현상슬라디딩  도어 작동시 잡음으로  입고됨원인도어 센터롤러부 청소 후...     N  N   N  N   
9570   C  브레이크 제동시 끄끅 끄끅 하는 이음과다점검내용및원인월일방문시 스프링 이음으...     N  N   N  N   

     target_predict test_category_predict  
9566              N                     O  
9567              N                     X  
9568              N                     N  
9569              N                     X  
9570              N                     O

# ◈ 결과검증

In [103]:
df2_test ['predict_diff'] = 0
df2_test ['predict_diff'] = (df2_test['대상 선정'] == df2_test['test_category_predict']) == 1
df2_test['predict_diff'] = df2_test['predict_diff'].astype(int)
df2_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
9566               W  2017     20171127         ZZ5         Q57   소음/이음_작동시   
9567               W  2015     20150824         ZZ5         Q57   소음/이음_작동시   
9568               W  2015     20150213         ZZ3         Q57   소음/이음_작동시   
9569               W  2016     20161021         ZZ3         Q57   소음/이음_작동시   
9570               W  2016     20161213         ZZ3         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
9566         KV11          CK     J5          TT        82130   
9567         KV11          YP     A9          TT        83940   
9568         KV11          YP     A9          TT        82210   
9569         KV11          YP     A9          TT        83940   
9570         KV11          YP     A9          CC        32800   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  \
9566   C  방지턱 통과시 좌우측 FR도어부근 찌그덕 고무 비비는 소음발생  웨더스트립 ...     N  N   N  N   
9567   C   우측 슬라이드도어 닫힘시 쁘윽 소음 발생점검 및 조치내용  점검결과 리어도...     N  N   N  N   
9568   C    상기차량은 운전석 윈도우 하강 시  끄윽 거리는 이음이 발생하여 입고 됨...     N  N   N  N   
9569   C  JXW현상슬라디딩  도어 작동시 잡음으로  입고됨원인도어 센터롤러부 청소 후...     N  N   N  N   
9570   C  브레이크 제동시 끄끅 끄끅 하는 이음과다점검내용및원인월일방문시 스프링 이음으...     N  N   N  N   

     target_predict test_category_predict  predict_diff  
9566              N                     O             0  
9567              N                     X             0  
9568              N                     N             1  
9569              N                     X             0  
9570              N                     O             0

In [104]:
diff_sum = df2_test['predict_diff'].sum()
diff_sum

423

In [105]:
y_predict_len = len(y_predict)
y_predict_len

1063

# ◈ 정확도

In [106]:
Acy = round(diff_sum/y_predict_len*100,1)
Acy

39.8